# Memory-Efficient Binary Neural Network (BNN) - 244×244 Images

This notebook extends the optimized 1-hidden layer BNN to 244×244 images. It includes progress bars via tqdm and comprehensive visualizations: ROC curves, Precision-Recall curves, training vs. validation accuracy, and epoch timing.

In [5]:
# Import Required Libraries
# %pip install torch torchvision matplotlib scikit-learn tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from sklearn.metrics import roc_curve, auc, precision_recall_curve
import time
import os
import random
import gc

# For reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available(): torch.cuda.manual_seed_all(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [6]:
# Memory Optimization Settings
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True,max_split_size_mb:32'

# Use moderate/aggressive as needed
def optimize_memory(mode='aggressive'):
    if mode == 'aggressive': return {'image_size':244, 'batch_size':8, 'hidden_size':256, 'embedding_size':512, 'num_hidden_layers':1, 'gradient_accumulation':4}
    else: return {'image_size':244, 'batch_size':16, 'hidden_size':320, 'embedding_size':640, 'num_hidden_layers':1, 'gradient_accumulation':2}

memory_config = optimize_memory('aggressive')
image_size = memory_config['image_size']
batch_size = memory_config['batch_size']
print(f"Configured for {image_size}x{image_size} images, batch size {batch_size}")

Configured for 244x244 images, batch size 8


In [7]:
# Load Dataset
# dataset_path = "/path/to/plant-disease-dataset"  # update path
dataset_path = "/home/dragoon/Downloads/MH-SoyaHealthVision An Indian UAV and Leaf Image Dataset for Integrated Crop Health Assessment/Soyabean_UAV-Based_Image_Dataset"  # update path
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])
full_dataset = datasets.ImageFolder(dataset_path, transform=transform)
class_names = full_dataset.classes
print(f"Classes: {class_names}")

# Split train/val/test (70/15/15)
total = len(full_dataset)
train_len = int(0.7*total)
val_len = int(0.15*total)
test_len = total - train_len - val_len
train_ds, val_ds, test_ds = torch.utils.data.random_split(full_dataset,[train_len,val_len,test_len], generator=torch.Generator().manual_seed(42))

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, pin_memory=True)

print(f"Train: {len(train_ds)} samples, Val: {len(val_ds)}, Test: {len(test_ds)}")

Classes: ['Healthy_Soyabean', 'Soyabean Semilooper_Pest_Attack', 'Soyabean_Mosaic', 'rust']
Train: 1989 samples, Val: 426, Test: 427


In [8]:
# Define Optimized BNN Model
class OptimizedBNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, embedding_size, num_hidden_layers=1, dropout_rate=0.35):
        super().__init__()
        # embedding
        self.embedding = nn.Sequential(
            nn.Linear(input_size, input_size//4), nn.ReLU(), nn.BatchNorm1d(input_size//4), nn.Dropout(dropout_rate),
            nn.Linear(input_size//4, embedding_size), nn.ReLU(), nn.BatchNorm1d(embedding_size)
        )
        self.input_binary = nn.Linear(embedding_size, hidden_size)
        self.hidden = nn.ModuleList([nn.Linear(hidden_size, hidden_size) for _ in range(num_hidden_layers)])
        self.output = nn.Linear(hidden_size, num_classes)
        self.dropout = nn.Dropout(dropout_rate)
    def forward(self,x):
        x = x.view(x.size(0),-1)
        x = self.embedding(x)
        x = torch.sign(self.input_binary(x))
        for layer in self.hidden:
            x = torch.sign(layer(x))
            x = self.dropout(x)
        return self.output(x)

input_size = 3*image_size*image_size
num_classes = len(class_names)
model = OptimizedBNN(input_size, memory_config['hidden_size'], num_classes, memory_config['embedding_size']).to(device)
print(model)

RuntimeError: [enforce fail at alloc_cpu.cpp:119] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 31900817664 bytes. Error code 12 (Cannot allocate memory)

In [ ]:
# Training Function with tqdm
from torch.cuda.amp import autocast, GradScaler
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)
scaler = GradScaler()

def train_validate(model, train_loader, val_loader, epochs):
    history = {'train_loss':[],'train_acc':[],'val_loss':[],'val_acc':[],'epoch_time':[]}
    # Early stopping parameters
    best_val_acc = 0.0
    no_improve = 0
    best_state = None

    for epoch in range(1, epochs+1):
        start = time.time()
        # TRAIN
        model.train(); running_loss=0; correct=0; total=0
        for x,y in tqdm(train_loader, desc=f"Epoch {epoch}/{epochs} [Train]"):  
            x,y = x.to(device),y.to(device)
            optimizer.zero_grad()
            with autocast(): outputs = model(x); loss = criterion(outputs,y)
            scaler.scale(loss).backward(); scaler.step(optimizer); scaler.update()
            running_loss += loss.item()*y.size(0)
            pred = outputs.argmax(1); correct += (pred==y).sum().item(); total+=y.size(0)
        train_loss = running_loss/total; train_acc=100*correct/total
        # VAL
        model.eval(); v_loss=0; v_corr=0; v_tot=0
        all_probs=[]; all_targets=[]
        with torch.no_grad():
            for x,y in tqdm(val_loader, desc=f"Epoch {epoch}/{epochs} [Val]"):  
                x,y = x.to(device),y.to(device)
                out = model(x); loss = criterion(out,y)
                v_loss += loss.item()*y.size(0)
                p = out.argmax(1); v_corr += (p==y).sum().item(); v_tot+=y.size(0)
                all_probs.append(F.softmax(out,1).cpu().numpy()); all_targets.append(y.cpu().numpy())
        val_loss = v_loss/v_tot; val_acc=100*v_corr/v_tot
        scheduler.step()
        # Early stopping: track best validation accuracy
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_state = model.state_dict()
            no_improve = 0
        else:
            no_improve += 1
        if 'early_stopping_patience' in locals() and no_improve >= early_stopping_patience:
            print(f"Early stopping at epoch {epoch}, best val_acc={best_val_acc:.2f}%")
            break

        et = time.time()-start
        history['train_loss'].append(train_loss); history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss); history['val_acc'].append(val_acc)
        history['epoch_time'].append(et)
        print(f"Epoch {epoch}: train_acc={train_acc:.2f}%, val_acc={val_acc:.2f}%, time={et:.2f}s")
    # Load best model weights if available
    if best_state is not None:
        model.load_state_dict(best_state)
        print(f"Loaded best model weights with val_acc={best_val_acc:.2f}%")

    return history, np.vstack(all_probs), np.concatenate(all_targets)

history, probs, targets = train_validate(model, train_loader, val_loader, epochs=20)

In [ ]:
# Plot Training History
plt.figure(figsize=(12,4))
plt.subplot(1,3,1)
plt.plot(history['train_loss'], label='Train Loss')
plt.plot(history['val_loss'], label='Val Loss'); plt.legend(); plt.title('Loss')
plt.subplot(1,3,2)
plt.plot(history['train_acc'], label='Train Acc')
plt.plot(history['val_acc'], label='Val Acc'); plt.legend(); plt.title('Accuracy')
plt.subplot(1,3,3)
plt.bar(range(1,len(history['epoch_time'])+1), history['epoch_time']); plt.title('Time/epoch'); plt.xlabel('Epoch'); plt.ylabel('Seconds')
plt.tight_layout(); plt.show()

In [ ]:
# ROC and Precision-Recall Curves
from sklearn.preprocessing import label_binarize

# Binarize targets
y_bin = label_binarize(targets, classes=list(range(num_classes)))
fpr, tpr, roc_auc = dict(), dict(), dict()
precision, recall, pr_auc = dict(), dict(), dict()
for i in range(num_classes):
    fpr[i], tpr[i], _ = roc_curve(y_bin[:,i], probs[:,i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    precision[i], recall[i], _ = precision_recall_curve(y_bin[:,i], probs[:,i])
    pr_auc[i] = auc(recall[i], precision[i])

# Plot ROC
plt.figure(figsize=(6,5))
for i in range(num_classes):
    plt.plot(fpr[i], tpr[i], label=f"{class_names[i]} (AUC={roc_auc[i]:.2f})")
plt.plot([0,1],[0,1],'k--'); plt.title('ROC Curves'); plt.xlabel('FPR'); plt.ylabel('TPR'); plt.legend(); plt.show()

# Plot PR
plt.figure(figsize=(6,5))
for i in range(num_classes):
    plt.plot(recall[i], precision[i], label=f"{class_names[i]} (AUC={pr_auc[i]:.2f})")
plt.title('Precision-Recall Curves'); plt.xlabel('Recall'); plt.ylabel('Precision'); plt.legend(); plt.show()

**End of Notebook: BNN244x244new.ipynb**

In [ ]:
# Save the trained model
import os
save_dir = 'results'
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, 'bnn_244x244_model_new.pt')
torch.save(model.state_dict(), save_path)
print(f"Model saved to {save_path}")